# Segmenting And Clustering Neighbourhoods In Toronto

### Part I - Neighbourhood Dataframe Creation

In [2]:
import pandas as pd # Importing Pandas

<em>**Reading a table from web by using pandas**</em>

In [3]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [4]:
raw_data = tables[0]

In [5]:
raw_data.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<em>**Filtering Borough those have values "Not assigned"**</em>

In [6]:
raw_data_2 = raw_data[raw_data['Borough'] != "Not assigned"]
raw_data_2.reset_index(drop=True, inplace=True)

<em>**Replacing a value of Neighbourhood that have value "Not assigned" with corresponding value of Borough**</em>

In [7]:
p = raw_data_2[raw_data_2['Neighbourhood'] == 'Not assigned'].index
xv = raw_data_2[raw_data_2['Neighbourhood'] == 'Not assigned'].values[0][1]

In [8]:
print("Index of \'Not assigned\':",p[0],"\nValue to be replaced with:",xv)

Index of 'Not assigned': 6 
Value to be replaced with: Queen's Park


In [10]:
raw_data_2.at[p,'Neighbourhood'] = xv

<em>**Bringing Neighbourhood dataframe in desired format, that is, all Neighbourhoods grouped by commas at Postal Code**</em>

In [11]:
def f(x):
     return pd.DataFrame.from_dict(dict(Postcode = x['Postcode'], 
                        Borough = x['Borough'], 
                        Neighbourhood = "%s" % ', '.join(x['Neighbourhood'])))

In [12]:
raw_data_3 = raw_data_2.groupby('Postcode').apply(f)

In [13]:
raw_data_3.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,"Rouge, Malvern"


<em>**Final shape of Neighbourhood dataframe**</em>

In [14]:
raw_data_3.shape

(211, 3)

### Part II - Importing Cordinates From Geospatial Data in Neighbourhood Dataframe

<em>**Getting Geospatial data in CSV format from given URL**</em>

In [15]:
url = "https://cocl.us/Geospatial_data"
geo_data = pd.read_csv(url)

In [16]:
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


<em>**Bringing Latitude and Longitude information from Geospatial data into Neighbourhood data**</em>

In [17]:
geo_data.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [18]:
dd = raw_data_3.set_index('Postcode').join(geo_data.set_index('Postcode'),on = 'Postcode', how = 'left')
dd.reset_index(inplace=True)

In [19]:
dd.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
9,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353


### Part III - Exploring And Clustering Neighbourhoods In Toronto

<em>**Formation of Toronto dataframe from Neighbourhood dataframe**</em>

In [20]:
toronto_data = dd[dd['Borough'].str.contains("Toronto")]

In [21]:
toronto_data.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
13,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
14,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
36,M4E,East Toronto,The Beaches,43.676357,-79.293031
37,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
41,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
42,M6G,Downtown Toronto,Christie,43.669542,-79.422564
49,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568


<em>**Getting geographical coordinates for Toronto**</em>

In [22]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [23]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<em>**Creating a map for segmenting and clustering neighbourhoods in Toronto**</em>

In [24]:
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | #################################

In [25]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
print("This is end of project")

This is end of project
